In [10]:
import requests

In [11]:
def judg_csv(judgment):
    from ltr.helpers.movies import get_movie
    judgedMovie = get_movie(movies='data/tmdb_ai_pow_search.json', tmdb_id=judgment.doc_id)
    return "{grade},'{title}',{keywords}".format(grade=judgment.grade,
                                                 title=judgedMovie['title'],
                                                 keywords=judgment.keywords)



In [12]:
from ltr.client.solr_client import SolrClient

client = SolrClient(host='http://localhost:8983/solr')

In [13]:
from ltr.judgments import Judgment

Judgment(grade=1, keywords='social network', doc_id=37799)

Judgment(grade=1,qid=1,keywords=social network,doc_id=37799,features=[],weight=1

In [14]:

mini_judg_list=[
    # for 'social network' query
    Judgment(grade=1, keywords='social network', doc_id='37799'), #The Social Network
    Judgment(grade=0, keywords='social network', doc_id='267752'), # #chicagoGirl
    Judgment(grade=0, keywords='social network', doc_id='38408'), # Life As We Know It
    Judgment(grade=0, keywords='social network', doc_id='28303'), # The Cheyenne Social Club
    
    # for 'star wars' query
    Judgment(grade=1, keywords='star wars', doc_id='11'), # star wars
    Judgment(grade=1, keywords='star wars', doc_id='1892'), # return of jedi
    Judgment(grade=0, keywords='star wars', doc_id='54138'),# Star Trek Into Darkness
    Judgment(grade=0, keywords='star wars', doc_id='85783'), # The Star
    Judgment(grade=0, keywords='star wars', doc_id='325553'), # Battlestar Galactica
]

for j in mini_judg_list:
    print(judg_csv(j))

1,'The Social Network',social network
0,'#chicagoGirl',social network
0,'Life As We Know It',social network
0,'The Cheyenne Social Club',social network
1,'Star Wars',star wars
1,'Return of the Jedi',star wars
0,'Star Trek Into Darkness',star wars
0,'The Star',star wars
0,'Battlestar Galactica',star wars


In [9]:
from ltr.judgments import judgments_open

with judgments_open('data/dummy_judgments.txt', 'w') as judgment_writer:
    for j in mini_judg_list:
        judgment_writer.write(j)

print(open('data/dummy_judgments.txt').read())

# qid:1: social network*1
# qid:2: star wars*1

1	qid:1	 # 37799	social network
0	qid:1	 # 267752	social network
0	qid:1	 # 38408	social network
0	qid:1	 # 28303	social network
1	qid:2	 # 11	star wars
1	qid:2	 # 1892	star wars
0	qid:2	 # 54138	star wars
0	qid:2	 # 85783	star wars
0	qid:2	 # 325553	star wars



In [14]:
mini_judg_list[0].features


[]

In [15]:
feature_set = [
    {
      "name" : "title_bm25",
      "store": "movies",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : { #q=title:({$keywords})
        "q" : "title:(${keywords})"
      }
    },
    {
      "name" : "overview_bm25",
      "store": "movies",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "overview:(${keywords})"
      }
    },
    {
      "name" : "vote_average",
      "store": "movies",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "{!func}vote_average"
}}]

requests.put('http://localhost:8983/solr/tmdb/schema/feature-store',
             json=feature_set)


<Response [500]>

In [16]:
from ltr.log import FeatureLogger
from ltr.judgments import judgments_open
from itertools import groupby

ftr_logger=FeatureLogger(client, index='tmdb', feature_set='movie')
for qid, query_judgments in groupby(mini_judg_list, key=lambda j: j.qid):
    ftr_logger.log_for_qid(judgments=query_judgments, 
                           qid=qid)
        
ftr_logger.logged

Searching tmdb [Status: 200]
Discarded 0 Keep 4
Searching tmdb [Status: 200]
Discarded 0 Keep 5


[Judgment(grade=1,qid=1,keywords=social network,doc_id=37799,features=[18.135925, 8.391596, 7.3],weight=1,
 Judgment(grade=0,qid=1,keywords=social network,doc_id=267752,features=[0.0, 13.237938, 7.0],weight=1,
 Judgment(grade=0,qid=1,keywords=social network,doc_id=38408,features=[0.0, 9.576859, 6.8],weight=1,
 Judgment(grade=0,qid=1,keywords=social network,doc_id=28303,features=[7.5430527, 6.839079, 6.4],weight=1,
 Judgment(grade=1,qid=2,keywords=star wars,doc_id=11,features=[14.951998, 0.0, 8.2],weight=1,
 Judgment(grade=1,qid=2,keywords=star wars,doc_id=1892,features=[0.0, 4.3300323, 8.0],weight=1,
 Judgment(grade=0,qid=2,keywords=star wars,doc_id=54138,features=[5.377082, 0.0, 7.3],weight=1,
 Judgment(grade=0,qid=2,keywords=star wars,doc_id=85783,features=[7.01165, 0.0, 7.5],weight=1,
 Judgment(grade=0,qid=2,keywords=star wars,doc_id=325553,features=[0.0, 0.0, 8.1],weight=1]

In [17]:
logging_solr_query = {
    "fl": "id,title,[features store=movie efi.keywords=\"social network\"]",
    'q': "id:37799 OR id:267752 id:38408 OR id:28303", #social network graded documents
    'rows': 10,
    'wt': 'json'  
}

resp = requests.post('http://localhost:8983/solr/tmdb/select',
                     data=logging_solr_query)

resp.json()

{'responseHeader': {'status': 0,
  'QTime': 4,
  'params': {'q': 'id:37799 OR id:267752 id:38408 OR id:28303',
   'fl': 'id,title,[features store=movie efi.keywords="social network"]',
   'rows': '10',
   'wt': 'json'}},
 'response': {'numFound': 4,
  'start': 0,
  'docs': [{'id': '38408',
    'title': ['Life As We Know It'],
    '[features]': 'title_bm25=0.0,overview_bm25=9.576859,vote_average=6.8'},
   {'id': '28303',
    'title': ['The Cheyenne Social Club'],
    '[features]': 'title_bm25=7.5430527,overview_bm25=6.839079,vote_average=6.4'},
   {'id': '37799',
    'title': ['The Social Network'],
    '[features]': 'title_bm25=18.135925,overview_bm25=8.391596,vote_average=7.3'},
   {'id': '267752',
    'title': ['#chicagoGirl'],
    '[features]': 'title_bm25=0.0,overview_bm25=13.237938,vote_average=7.0'}]}}

In [28]:
# Save off features for qid=1
CURR_QID=1

solr_json = resp.json()
doc_id_to_features = {}

# Map Doc Id => Features
for doc in solr_json['response']['docs']:
    # Parse '[features] array', ie
    # title_bm25=0.0,overview_bm25=13.237938,vote_average=7.0'
    features = doc['[features]']
    features = features.split(',')
    features = [float(ftr.split('=')[1]) for ftr in features]
    
    doc_id_to_features[doc['id']] = features

# Save in correct judgment
for judgment in mini_judg_list:
    if judgment.qid == CURR_QID:
        try:
            judgment.features = doc_id_to_features[judgment.doc_id]
        except KeyError:
            pass
    

{'38408': [0.0, 9.576859, 6.8], '28303': [7.5430527, 6.839079, 6.4], '37799': [18.135925, 8.391596, 7.3], '267752': [0.0, 13.237938, 7.0]}


In [30]:
from ltr.log import FeatureLogger
from ltr.judgments import judgments_open
from itertools import groupby

ftr_logger=FeatureLogger(client, index='tmdb', feature_set='movie')
for qid, query_judgments in groupby(mini_judg_list, key=lambda j: j.qid):
    ftr_logger.log_for_qid(judgments=query_judgments, 
                           qid=qid)
        
ftr_logger.logged

Searching tmdb [Status: 200]
Discarded 0 Keep 4
Searching tmdb [Status: 200]
Discarded 0 Keep 5


[Judgment(grade=1,qid=1,keywords=social network,doc_id=37799,features=[18.135925, 8.391596, 7.3],weight=1,
 Judgment(grade=0,qid=1,keywords=social network,doc_id=267752,features=[0.0, 13.237938, 7.0],weight=1,
 Judgment(grade=0,qid=1,keywords=social network,doc_id=38408,features=[0.0, 9.576859, 6.8],weight=1,
 Judgment(grade=0,qid=1,keywords=social network,doc_id=28303,features=[7.5430527, 6.839079, 6.4],weight=1,
 Judgment(grade=1,qid=2,keywords=star wars,doc_id=11,features=[14.951998, 0.0, 8.2],weight=1,
 Judgment(grade=1,qid=2,keywords=star wars,doc_id=1892,features=[0.0, 4.3300323, 8.0],weight=1,
 Judgment(grade=0,qid=2,keywords=star wars,doc_id=54138,features=[5.377082, 0.0, 7.3],weight=1,
 Judgment(grade=0,qid=2,keywords=star wars,doc_id=85783,features=[7.01165, 0.0, 7.5],weight=1,
 Judgment(grade=0,qid=2,keywords=star wars,doc_id=325553,features=[0.0, 0.0, 8.1],weight=1]

In [32]:
from ltr import download

judgments='http://es-learn-to-rank.labs.o19s.com/title_judgments_binary.txt'
download([judgments], dest='data/')

ftr_logger=FeatureLogger(client, index='tmdb', feature_set='movie')

with judgments_open('data/title_judgments.txt') as judgment_list:
    for qid, query_judgments in groupby(judgment_list, key=lambda j: j.qid):
        ftr_logger.log_for_qid(judgments=query_judgments, 
                               qid=qid,
                               keywords=judgment_list.keywords(qid))
        
ftr_logger.logged

data/title_judgments_binary.txt already exists
Recognizing 40 queries...
Searching tmdb [Status: 200]
Discarded 0 Keep 41
Searching tmdb [Status: 200]
Discarded 0 Keep 41
Searching tmdb [Status: 200]
Discarded 0 Keep 39
Searching tmdb [Status: 200]
Discarded 0 Keep 28
Searching tmdb [Status: 200]
Discarded 0 Keep 33
Searching tmdb [Status: 200]
Discarded 0 Keep 39
Searching tmdb [Status: 200]
Discarded 0 Keep 35
Searching tmdb [Status: 200]
Discarded 0 Keep 38
Searching tmdb [Status: 200]
Discarded 0 Keep 35
Searching tmdb [Status: 200]
Discarded 0 Keep 31
Searching tmdb [Status: 200]
Discarded 0 Keep 28
Searching tmdb [Status: 200]
Discarded 0 Keep 34
Searching tmdb [Status: 200]
Discarded 0 Keep 31
Searching tmdb [Status: 200]
Discarded 0 Keep 30
Searching tmdb [Status: 200]
Discarded 0 Keep 35
Searching tmdb [Status: 200]
Discarded 0 Keep 31
Searching tmdb [Status: 200]
Discarded 0 Keep 31
Searching tmdb [Status: 200]
Discarded 0 Keep 35
Searching tmdb [Status: 200]
Discarded 0 Keep

[Judgment(grade=4,qid=1,keywords=rambo,doc_id=7555,features=[13.038148, 11.173398, 6.5],weight=1,
 Judgment(grade=3,qid=1,keywords=rambo,doc_id=1370,features=[11.056428, 12.652582, 6.0],weight=1,
 Judgment(grade=3,qid=1,keywords=rambo,doc_id=1369,features=[7.593794, 10.758981, 6.5],weight=1,
 Judgment(grade=2,qid=1,keywords=rambo,doc_id=13258,features=[0.0, 10.096009, 6.7],weight=1,
 Judgment(grade=4,qid=1,keywords=rambo,doc_id=1368,features=[0.0, 11.867074, 7.4],weight=1,
 Judgment(grade=1,qid=1,keywords=rambo,doc_id=31362,features=[0.0, 8.33506, 6.0],weight=1,
 Judgment(grade=1,qid=1,keywords=rambo,doc_id=61410,features=[0.0, 4.6697874, 6.9],weight=1,
 Judgment(grade=0,qid=1,keywords=rambo,doc_id=319074,features=[0.0, 0.0, 6.0],weight=1,
 Judgment(grade=0,qid=1,keywords=rambo,doc_id=10296,features=[0.0, 0.0, 6.4],weight=1,
 Judgment(grade=0,qid=1,keywords=rambo,doc_id=35868,features=[0.0, 0.0, 5.4],weight=1,
 Judgment(grade=0,qid=1,keywords=rambo,doc_id=131457,features=[0.0, 0.0, 6.3